In [1]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.DataIO import DataIO
from skopt.space import Real, Integer, Categorical
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender,MultiThreadSLIM_SLIMElasticNetRecommender



columns_name = ['user_id','item_id','impression_list','data']

dataset_l = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_length.csv', sep=',')
dataset_t = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_type.csv', sep=',')
dataset = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/interactions_and_impressions.csv', sep=',')
test_users = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_target_users_test.csv',sep = ',')
dataset.columns = columns_name

c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
from Data_manager.Build_Matrix.Data_import import build_URM_ICM,build_URM_impression,build_URM_ICM_onlyURM_item
URM,ICM=build_URM_ICM_onlyURM_item(dataset, dataset_t, dataset_l)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.8)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 349 ( 0.8%) Users that have less than 1 test interactions


In [4]:
import os

output_folder_path = "../result_experiments/4_SLIM_py"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 20  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [5]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "epochs": Categorical([50]), # because I want one specific value which is the max
    "sgd_mode": Categorical(["sgd", "adagrad", "adam"]),
    "topK": Integer(50, 500),
    "lambda_i": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
    "lambda_j": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
    "learning_rate": Real(low = 1e-3, high = 1e-1, prior = 'log-uniform'),
}

In [6]:
earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [7]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = SLIM_BPR_Cython

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation)

In [12]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},    # Additiona hyperparameters for the fit function
    EARLYSTOPPING_KEYWORD_ARGS=earlystopping_keywargs,
)

In [13]:
hyperparameterSearch.search(recommender_input_args,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'epochs': 50, 'sgd_mode': 'adam', 'topK': 209, 'lambda_i': 0.0014332207745243535, 'lambda_j': 0.00024680054647576523, 'learning_rate': 0.095989768617544}
SearchBayesianSkopt: Config 0 Exception. Config: {'epochs': 50, 'sgd_mode': 'adam', 'topK': 209, 'lambda_i': 0.0014332207745243535, 'lambda_j': 0.00024680054647576523, 'learning_rate': 0.095989768617544} - Exception: Traceback (most recent call last):
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 449, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 317, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparamete

Traceback (most recent call last):
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 449, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 317, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 288, in _fit_model
    recommender_instance.fit(*self.recommender_input_args.FIT_POSITIONAL_ARGS,
  File "c:\Users\loren\Documents\GitHub\RecSys\Recommenders\SLIM\Cython\SLIM_BPR_Cython.py", line 81, in fit
    from Recommenders.SLIM.Cython.SLIM_BPR_Cython_Epoch import SLIM_BPR_Cython_Epoch
ModuleNotFoundError: No module named 'Recommenders.SLIM.Cython.SLIM_BPR_Cytho

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.1140
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 3 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'epochs': 50, 'sgd_mode': 'adagrad', 'topK': 420, 'lambda_i': 9.759884142588257e-05, 'lambda_j': 2.081204057196986e-05, 'learning_rate': 0.005487780471681143}
SearchBayesianSkopt: Config 2 Exception. Config: {'epochs': 50, 'sgd_mode': 'adagrad', 'topK': 420, 'lambda_i': 9.759884142588257e-05, 'lambda_j': 2.081204057196986e-05, 'learning_rate': 0.005487780471681143} - Exception: Traceback (most recent call last):
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 449, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "c:\Users\loren\Documents\GitHub\RecSys\Hyperparameter

Traceback (most recent call last):
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 449, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 317, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 288, in _fit_model
    recommender_instance.fit(*self.recommender_input_args.FIT_POSITIONAL_ARGS,
  File "c:\Users\loren\Documents\GitHub\RecSys\Recommenders\SLIM\Cython\SLIM_BPR_Cython.py", line 81, in fit
    from Recommenders.SLIM.Cython.SLIM_BPR_Cython_Epoch import SLIM_BPR_Cython_Epoch
ModuleNotFoundError: No module named 'Recommenders.SLIM.Cython.SLIM_BPR_Cytho

Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.1160
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 5 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'epochs': 50, 'sgd_mode': 'adam', 'topK': 138, 'lambda_i': 0.0003699405377326741, 'lambda_j': 0.00014642531004535066, 'learning_rate': 0.0077798723050251295}
SearchBayesianSkopt: Config 4 Exception. Config: {'epochs': 50, 'sgd_mode': 'adam', 'topK': 138, 'lambda_i': 0.0003699405377326741, 'lambda_j': 0.00014642531004535066, 'learning_rate': 0.0077798723050251295} - Exception: Traceback (most recent call last):
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 449, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTu

Traceback (most recent call last):
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 449, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 317, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "c:\Users\loren\Documents\GitHub\RecSys\HyperparameterTuning\SearchAbstractClass.py", line 288, in _fit_model
    recommender_instance.fit(*self.recommender_input_args.FIT_POSITIONAL_ARGS,
  File "c:\Users\loren\Documents\GitHub\RecSys\Recommenders\SLIM\Cython\SLIM_BPR_Cython.py", line 81, in fit
    from Recommenders.SLIM.Cython.SLIM_BPR_Cython_Epoch import SLIM_BPR_Cython_Epoch
ModuleNotFoundError: No module named 'Recommenders.SLIM.Cython.SLIM_BPR_Cytho

SearchBayesianSkopt: Search interrupted. No valid config was found during the initial random initialization



In [10]:
import pandas as pd
recommender = SLIM_BPR_Cython(URM_all.tocsr()) # <-----
at = 10

recommender.fit(epochs = 210, sgd_mode = 'sgd', topK = 800, lambda_i = 1.0737760587059938e-05, lambda_j = 0.01,
                learning_rate = 0.009522871552947681)
user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
sep=",",
dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, cutoff=at)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


	# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
	# submission

from datetime import datetime

now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)



NameError: name 'URM_all' is not defined